In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import torch
import math
import pickle


In [4]:
#split the dataset

dataset = pickle.load(open('datasets.pickle','rb'))

batchsize = 256
#train_x,test_x,train_y,test_y = train_test_split(torch.tensor(X),torch.tensor(Y),test_size=0.2,stratify=torch.tensor(Y))
#train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
#trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainset = dataset[0]
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)

#valset = torch.utils.data.TensorDataset(val_x,val_y)
valset = dataset[1]
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)

#testset = torch.utils.data.TensorDataset(test_x,test_y)

testset = dataset[2]
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)



print(trainset[0][0][:4],trainset[0][1].shape)

input_dim = 4
output_dim = trainset[0][1].shape[0]

tensor([ 209, 5287,    5,    2]) torch.Size([5911])


In [5]:
!pip install pytorch-lightning
import pytorch_lightning as pl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 9.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [30]:

#define the model




class MLP(pl.LightningModule):

	def __init__(self, in_channels, hidden_channels, out_channels):
		super().__init__()
		self.emb1 = torch.nn.Embedding(100000, 32)
		self.emb2 = torch.nn.Embedding(100000, 32)
		self.timeday = torch.nn.Linear(2, 32)
		self.class1 = torch.nn.Linear(96, hidden_channels)
		self.class2 = torch.nn.Linear(hidden_channels, 32)
		self.class3 = torch.nn.Linear(32, 16)
		self.class4 = torch.nn.Linear(16, out_channels)

	def forward(self, data):
		user = self.emb1(data[:, 0])
		loc = self.emb2(data[:, 1])
		hour = data[:, 2]
		day = data[:, 3]
		timeday = torch.stack([hour, day], dim=-1)
		timeday = self.timeday(timeday.float())
		x = torch.cat((user, loc, timeday), dim=1)
		x = self.class1(x).relu()
		x = self.class2(x).relu()
		x = self.class3(x).relu()
		x = self.class4(x)
		return x

	def training_step(self, batch, batch_idx):
		input = batch[0]
		target = batch[1]
		output = self(input[:, :4])
		loss = torch.nn.functional.cross_entropy(output, target)
		self.log('train_loss', loss)
		return loss

	def validation_step(self, batch, batch_idx):
		input = batch[0]
		target = batch[1]
		output = self(input[:, :4])
		val_loss = torch.nn.functional.cross_entropy(output, target)
		acc_at_k = accuracy_at_k(output, target, k=5)
		mrr = mean_reciprocal_rank(output, target)

		self.log('val_loss', val_loss)
		self.log('val_acc_at_5', acc_at_k)
		self.log('val_mrr', mrr)
		self.log('val_loss', val_loss)
		return val_loss


	def configure_optimizers(self):
		optimizer = torch.optim.Adam(self.parameters(), lr= 0.001, weight_decay=5e-4)
		return optimizer

	def accuracy_at_k(y_pred, y_true, k=5):
			_, top_k = y_pred.topk(k, dim=1)
			_,label = y_true.topk(1, dim=1)

			print(top_k)
			print(label)

			correct = top_k.eq(label.view(-1, 1).expand_as(top_k))
			#print(label.view(-1, 1).expand_as(top_k))
			acc_at_k = correct.float().sum(dim=1).mean()
			print(acc_at_k)
			return acc_at_k

	def mean_reciprocal_rank(y_pred, y_true):
			_, rank = y_pred.sort(dim=1, descending=True)
			rank = rank.argsort(dim=1)
			_,label = y_true.topk(1, dim=1)
			#print(label.view(-1, 1))
			rr = (1.0 / (rank.gather(1, label.view(-1, 1).long()) + 1)).mean()
			print(rr)
			return rr


In [31]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(f"Using device: {device}")
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

numepoch = 500


model = MLP(4,64,dataset[0][1][1].shape[0])
criterion = torch.nn.CrossEntropyLoss()
trainer = pl.Trainer(max_epochs=numepoch, log_every_n_steps=1,callbacks=[
          ModelCheckpoint(monitor='train_loss'),
          EarlyStopping(monitor='train_loss', patience=10)
      ])
trainer.fit(model, trainloader)

#test_results = trainer.validation(model)
#print(test_results)

print(model)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type      | Params
--------------------------------------
0 | emb1    | Embedding | 3.2 M 
1 | emb2    | Embedding | 3.2 M 
2 | timeday | Linear    | 96    
3 | class1  | Linear    | 6.2 K 
4 | class2  | Linear    | 2.1 K 
5 | class3  | Linear    | 528   
6 | class4  | Linear    | 100 K 
--------------------------------------
6.5 M     Trainable param

Training: |          | 0/? [00:00<?, ?it/s]

MLP(
  (emb1): Embedding(100000, 32)
  (emb2): Embedding(100000, 32)
  (timeday): Linear(in_features=2, out_features=32, bias=True)
  (class1): Linear(in_features=96, out_features=64, bias=True)
  (class2): Linear(in_features=64, out_features=32, bias=True)
  (class3): Linear(in_features=32, out_features=16, bias=True)
  (class4): Linear(in_features=16, out_features=5911, bias=True)
)


In [ ]:
import matplotlib.pyplot as plt
#print CE loss evolution
ind = [i for i in range(len(losses))]
losses = [loss.cpu().detach().numpy() for loss in losses]
plt.plot(ind,losses)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

#print loss evolution

ind = [i for i in range(len(losses))]
losses = [loss.cpu().detach().numpy() for loss in losses]
plt.plot(ind,losses)
plt.show()